In [196]:
import pandas as pd

# loading all datasets
url_client_profiles = 'data/df_final_demo.txt'
url_digital_footprints1 = 'data/df_final_web_data_pt_1.txt'
url_digital_footprints2 = 'data/df_final_web_data_pt_2.txt'
url_experiment_roster = 'data/df_final_experiment_clients.txt'

df1 = pd.read_csv(url_digital_footprints1)
df2 = pd.read_csv(url_digital_footprints2)

# imported dataframes to work with
df_client_profiles = pd.read_csv(url_client_profiles)
df_exp_roster = pd.read_csv(url_experiment_roster)
# merged footprint files
df_footprints = pd.concat([df1, df2])

In [198]:
# cleaning the datasets
df_client_profiles.rename(columns={'clnt_tenure_yr': 'client_tenure_years', 'clnt_tenure_mnth': 'client_tenure_months', 'clnt_age': 'client_age', 'gendr': 'gender', 'num_accts': 'num_accounts', 'bal': 'balance', 'calls_6_mnth': 'calls_6months', 'logons_6_mnth': 'logins_6months'}, inplace=True)
df_client_profiles_cleaned = df_client_profiles.dropna(subset=["client_tenure_years", "client_tenure_months", "client_age", "gender", "num_accounts", "balance", "calls_6months", "logins_6months"], how="all")
df_exp_roster_cleaned = df_exp_roster.dropna(subset=["Variation"], how="all")
df_footprints_cleaned = df_footprints.dropna(subset=["client_id", "visitor_id", "visit_id", "process_step", "date_time"], how="all")

df_client_profiles_cleaned[df_client_profiles_cleaned['gender']=='U']
df_client_profiles_cleaned['gender'] = df_client_profiles_cleaned['gender'].apply(lambda x: 'U' if x == 'X' else x)
df_client_profiles_cleaned['gender'].fillna('U')
df_footprints_cleaned.drop_duplicates(subset=['client_id', 'visit_id', 'date_time'])

/var/folders/gq/19wnck6n1lggc_91r4nzbjk80000gn/T/ipykernel_13194/4035102539.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_client_profiles_cleaned['gender'] = df_client_profiles_cleaned['gender'].apply(lambda x: 'U' if x == 'X' else x)


,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [200]:
# joining footprints and experiment roster datasets
df_footprints_cleaned.set_index('client_id', inplace=True)
df_exp_roster_cleaned.set_index('client_id', inplace=True)

joined_df = df_footprints_cleaned.join(df_exp_roster_cleaned, how='inner')
joined_df['date_time'] = pd.to_datetime(joined_df['date_time'])
df_sorted = joined_df.sort_values(by=['Variation', 'visit_id', 'date_time'])

In [202]:
# Calculating Completion Rate: 
vistor_df = df_sorted[df_sorted["process_step"] == "confirm"]
vistor_df['visit_id'].nunique()

df_sorted["visit_id"].nunique()

completion_rate = round(
    (vistor_df['visit_id'].nunique() / df_sorted["visit_id"].nunique()) * 100, 2
)

In [204]:
# Calculating Time Spent On Each Step

# Ensure 'date_time' is in datetime format
df_sorted['date_time'] = pd.to_datetime(df_sorted['date_time'])

# Sort by visit_id and date_time to ensure correct time difference calculation
df_sorted = df_sorted.sort_values(by=['visit_id', 'date_time'])

# Calculate time spent on each step (difference between consecutive timestamps within each visit)
df_sorted['time_spent'] = df_sorted.groupby('visit_id')['date_time'].diff().dt.total_seconds()

# Compute the average time spent per process_step
average_time_per_step = df_sorted.groupby('process_step')['time_spent'].mean()

# Merge the average time back into the original dataframe as a new column
df_sorted['time_each_step'] = df_sorted['process_step'].map(average_time_per_step)

# Round to 2 decimal places for readability
df_sorted['time_each_step'] = df_sorted['time_each_step'].round(2)

df_sorted

,visitor_id,visit_id,process_step,date_time,Variation,time_spent,time_each_step
client_id,,,,,,,
3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,Test,NaN,128.79
3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,Test,52.0,128.79
7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,Test,NaN,150.59
7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,Test,16.0,39.98
7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,Test,9.0,43.82
...,...,...,...,...,...,...,...
6627522,730634087_44272418812,999988789_76411676596_272843,start,2017-04-21 23:49:11,Test,NaN,150.59
6627522,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:49:22,Test,11.0,39.98
6627522,730634087_44272418812,999988789_76411676596_272843,step_2,2017-04-21 23:50:16,Test,54.0,43.82


In [ ]:
### Comparing completion rate KPI between groups ###

# Calculating Completion Rate for TEST group
vistor_df = df_test[df_test["process_step"] == "confirm"]
vistor_df['visit_id'].nunique()

df_test["visit_id"].nunique()

test_group_completion_rate = round(
    (vistor_df['visit_id'].nunique() / df_test["visit_id"].nunique()) * 100, 2
)

test_group_completion_rate

In [ ]:
# Calculating Completion Rate for CONTROL group
vistor_df = df_control[df_control["process_step"] == "confirm"]
vistor_df['visit_id'].nunique()

df_control["visit_id"].nunique()

control_group_completion_rate = round(
    (vistor_df['visit_id'].nunique() / df_control["visit_id"].nunique()) * 100, 2
)

control_group_completion_rate

print(f"The completion rate for the test group is {test_group_completion_rate}, while the completion rate for the control group is {control_group_completion_rate}")

In [224]:
# error rates calculation
possible_previous_steps = {
    'start' : None,
    'step_1' : 'start',
    'step_2' : 'step_1',
    'step_3' : 'step_2',
    'confirm' : 'step_3'
}

def check_error(df, possible_previous_steps):
    valid_count = 0
    error_counts = {'Control': 0, 'Test': 0}
    
    for (visit_id, variation), group in df.groupby(['visit_id', 'Variation']):
        previous_step = None

        for _, row in group.iterrows():  # Iterate over rows correctly
            step = row['process_step']
            #print(f"\nCurrent visit_id: {visit_id}")
                
            if previous_step is not None:
                if previous_step == step:
                    #print(f'Valid: {previous_step} to {step} is possible.')
                    valid_count += 1
                else:
                    expected_previous = possible_previous_steps.get(step)
                    if previous_step != expected_previous:
                        #print(f'ERROR: Switch from {previous_step} to {step} is not valid.')
                        error_counts[variation] += 1
                    else:
                        #print(f'Valid: {previous_step} to {step} is possible.')
                        valid_count += 1
            previous_step = step
    return error_counts

check_error(df_sorted, possible_previous_steps)

{'Control': 10053, 'Test': 17135}

In [ ]:
df_client_profiles_cleaned.groupby(['gender'])['client_age'].agg(['mean', 'median', 'min', 'max', 'count'])
df_client_profiles_cleaned.groupby(['gender'])['client_tenure_years'].agg(['mean', 'median', 'min', 'max', 'count'])
df_client_profiles_cleaned.groupby(['gender'])['balance'].agg(['mean', 'median', 'min', 'max', 'count'])

# activity score calculation
df_client_profiles_cleaned['activity_score'] = df_client_profiles_cleaned.iloc[:,8] + (df_client_profiles_cleaned.iloc[:,7]*0.5) + (df_client_profiles_cleaned.iloc[:,6]*0.00001)
df_client_profiles_cleaned.sort_values('activity_score', ascending=False)

#login and call scores
high_login_threshold = df_client_profiles_cleaned['logins_6months'].quantile(0.8)
low_login_threshold = df_client_profiles_cleaned['logins_6months'].quantile(0.2)
high_call_threshold = df_client_profiles_cleaned['calls_6months'].quantile(0.8)
low_call_threshold = df_client_profiles_cleaned['calls_6months'].quantile(0.2)

def segment_customer(logins):
    if logins >= high_login_threshold:
        return "Highly Active"
    elif logins <= low_login_threshold:
        return "Inactive"
    else:
        return "Moderate"

def segment_customer(calls):
    if calls >= high_call_threshold:
        return "Highly Active"
    elif calls <= low_call_threshold:
        return "Inactive"
    else:
        return "Moderate"

df_client_profiles_cleaned["login_activity"] = df_client_profiles_cleaned["logins_6months"].apply(segment_customer)
df_client_profiles_cleaned["calls_activity"] = df_client_profiles_cleaned["calls_6months"].apply(segment_customer)

In [ ]:
login_activity_counts = df_client_profiles_cleaned["login_activity"].value_counts()
print(login_activity_counts)
df_client_profiles_cleaned.groupby(["login_activity", "gender"])[["client_age", "balance", "client_tenure_years"]].mean()

In [ ]:
calls_activity_counts = df_client_profiles_cleaned["calls_activity"].value_counts()
print(calls_activity_counts)
df_client_profiles_cleaned.groupby(["calls_activity", "gender"])[["client_age", "balance", "client_tenure_years"]].mean()